In [1]:
"""
Inference Script
"""
import json
import os
import torch
import tqdm
from shutil import copyfile


# Detectron imports|
from detectron2.engine import launch
from detectron2.data import build_detection_test_loader, build_detection_train_loader, MetadataCatalog

# Project imports
import core.datasets.metadata as metadata

from core.setup import setup_config, setup_arg_parser
from offline_evaluation import compute_average_precision, compute_probabilistic_metrics, compute_calibration_errors
from probabilistic_inference.probabilistic_inference import build_predictor
from probabilistic_inference.inference_utils import instances_to_json

from baal.active import FileDataset, ActiveLearningDataset

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

** fvcore version of PathManager will be deprecated soon. **
** Please migrate to the version in iopath repo. **
https://github.com/facebookresearch/iopath 

** fvcore version of PathManager will be deprecated soon. **
** Please migrate to the version in iopath repo. **
https://github.com/facebookresearch/iopath 



In [2]:
import detectron2.utils.comm as comm
from detectron2.checkpoint import DetectionCheckpointer
from detectron2.data import build_detection_test_loader, build_detection_train_loader
from detectron2.engine import DefaultTrainer, launch
from detectron2.evaluation import COCOEvaluator, DatasetEvaluators, verify_results

In [3]:
##setup inference args, this also contains all the training args
arg_parser = setup_arg_parser()
args = arg_parser.parse_args("")
# Support single gpu inference only.
args.num_gpus = 1
args.dataset_dir = '/public-dataset/BDD/bdd100k'
args.test_dataset = 'bdd_val'
args.config_file = '/home/jefferson.keh/AL_CVPR/src/configs/BDD-Detection/retinanet/retinanet_R_50_FPN_1x_reg_cls_var_dropout.yaml'
args.inference_config = '/home/jefferson.keh/AL_CVPR/src/configs/Inference/bayes_od_mc_dropout.yaml'
print("Command Line Args:", args)

Command Line Args: Namespace(config_file='/home/jefferson.keh/AL_CVPR/src/configs/BDD-Detection/retinanet/retinanet_R_50_FPN_1x_reg_cls_var_dropout.yaml', dataset_dir='/public-dataset/BDD/bdd100k', dist_url='tcp://127.0.0.1:50153', eval_only=False, inference_config='/home/jefferson.keh/AL_CVPR/src/configs/Inference/bayes_od_mc_dropout.yaml', iou_correct=0.7, iou_min=0.1, machine_rank=0, min_allowed_score=0.0, num_gpus=1, num_machines=1, opts=[], random_seed=0, resume=False, test_dataset='bdd_val')


In [4]:
cfg = setup_config(args, random_seed=1234)

Loading config /home/jefferson.keh/AL_CVPR/src/configs/BDD-Detection/retinanet/../../Base-RetinaNet.yaml with yaml.unsafe_load. Your machine may be at risk if the file contains malicious content.
Config '/home/jefferson.keh/AL_CVPR/src/configs/Inference/bayes_od_mc_dropout.yaml' has no VERSION. Assuming it to be compatible with latest v2.


[02/19 16:49:51 detectron2]: Rank of current process: 0. World size: 1
[02/19 16:49:52 detectron2]: Environment info:
----------------------  ---------------------------------------------------------------------------------------
sys.platform            linux
Python                  3.8.5 (default, Sep  4 2020, 07:30:14) [GCC 7.3.0]
numpy                   1.19.5
detectron2              0.3 @/home/jefferson.keh/.conda/envs/pod_jeff/lib/python3.8/site-packages/detectron2
Compiler                GCC 7.3
CUDA compiler           CUDA 10.2
detectron2 arch flags   3.7, 5.0, 5.2, 6.0, 6.1, 7.0, 7.5
DETECTRON2_ENV_MODULE   <not set>
PyTorch                 1.7.1 @/home/jefferson.keh/.conda/envs/pod_jeff/lib/python3.8/site-packages/torch
PyTorch debug build     False
GPU available           True
GPU 0,1,2,3             Tesla V100-SXM2-32GB (arch=7.0)
CUDA_HOME               /usr/local/cuda
Pillow                  8.1.0
torchvision             0.8.2 @/home/jefferson.keh/.conda/envs/pod_jeff/lib/

[02/19 16:49:52 detectron2]: Full config saved to /home/jefferson.keh/AL_CVPR/data/BDD-Detection/retinanet/retinanet_R_50_FPN_1x_reg_cls_var_dropout/random_seed_1234/config.yaml


In [5]:
from detectron2.data import get_detection_dataset_dicts
from detectron2.data import build_detection_train_loader, build_detection_test_loader

In [6]:
dataset = get_detection_dataset_dicts(        
    cfg.DATASETS.TRAIN,
    filter_empty=cfg.DATALOADER.FILTER_EMPTY_ANNOTATIONS,
    min_keypoints=cfg.MODEL.ROI_KEYPOINT_HEAD.MIN_KEYPOINTS_PER_IMAGE
    if cfg.MODEL.KEYPOINT_ON
    else 0,
    proposal_files=cfg.DATASETS.PROPOSAL_FILES_TRAIN if cfg.MODEL.LOAD_PROPOSALS else None,
)

[02/19 16:49:56 d2.data.datasets.coco]: Loading /public-dataset/BDD/bdd100k/labels/train_coco_format.json takes 4.31 seconds.
[02/19 16:49:57 d2.data.datasets.coco]: Loaded 69863 images in COCO format from /public-dataset/BDD/bdd100k/labels/train_coco_format.json
[02/19 16:50:00 d2.data.build]: Removed 458 images with no usable annotations. 69405 images left.
[02/19 16:50:01 d2.data.build]: Distribution of instances among all 7 categories:
|  category  | #instances   |  category  | #instances   |  category  | #instances   |
|:----------:|:-------------|:----------:|:-------------|:----------:|:-------------|
|    car     | 713211       |    bus     | 11672        |   truck    | 29971        |
|   person   | 91349        |   rider    | 4517         |    bike    | 7210         |
|   motor    | 3002         |            |              |            |              |
|   total    | 860932       |            |              |            |              |


In [7]:
from torch.utils.data import Subset
import numpy as np

np.random.seed(0)
indices = np.random.choice(len(dataset), 2000, replace=False)
subsample_dataset = Subset(dataset, indices)

In [8]:
print(indices)

[40764 47959 36938 ... 16084 65627 27717]


In [9]:
dataset = ActiveLearningDataset(dataset)
subsample_dataset = ActiveLearningDataset(subsample_dataset)

In [10]:
print(dataset.pool.__len__())
print(subsample_dataset.pool.__len__())

69405
2000


In [11]:
print(dataset.pool[40764]['file_name'])
print(subsample_dataset.pool[0]['file_name'])

/public-dataset/BDD/bdd100k/images/100k/train/68eae8d4-6c437d9d.jpg
/public-dataset/BDD/bdd100k/images/100k/train/68eae8d4-6c437d9d.jpg


In [12]:
print(subsample_dataset.__len__())
print(subsample_dataset.pool.__len__())
print(subsample_dataset.pool[0]['file_name'])
subsample_dataset.label(0)
print(subsample_dataset.__len__())
print(subsample_dataset.pool.__len__())
print(subsample_dataset.pool[0]['file_name'])

0
2000
/public-dataset/BDD/bdd100k/images/100k/train/68eae8d4-6c437d9d.jpg
1
1999
/public-dataset/BDD/bdd100k/images/100k/train/7b583008-124b6809.jpg


In [13]:
train_data_loader = build_detection_train_loader(cfg, dataset=subsample_dataset.pool)

[02/19 16:50:01 d2.data.dataset_mapper]: [DatasetMapper] Augmentations used in training: [ResizeShortestEdge(short_edge_length=(720,), max_size=1333, sample_style='choice'), RandomFlip()]
[02/19 16:50:01 d2.data.build]: Using training sampler TrainingSampler


In [14]:
train_data_loader_iter = iter(train_data_loader)

In [15]:
print(next(train_data_loader_iter))

[{'file_name': '/public-dataset/BDD/bdd100k/images/100k/train/05296fa8-248ff695.jpg', 'height': 720, 'width': 1280, 'image_id': 1935, 'image': tensor([[[ 1,  1,  1,  ...,  2,  2,  2],
         [ 1,  1,  1,  ...,  2,  2,  2],
         [ 1,  1,  1,  ...,  2,  2,  2],
         ...,
         [ 2,  2,  2,  ...,  5,  5,  5],
         [ 2,  2,  2,  ...,  5,  5,  5],
         [ 2,  2,  2,  ...,  5,  5,  5]],

        [[ 1,  1,  1,  ...,  5,  5,  5],
         [ 1,  1,  1,  ...,  5,  5,  5],
         [ 1,  1,  1,  ...,  5,  5,  5],
         ...,
         [12, 12, 12,  ..., 16, 16, 16],
         [12, 12, 12,  ..., 16, 16, 16],
         [12, 12, 12,  ..., 16, 16, 16]],

        [[ 1,  1,  1,  ...,  9,  9,  9],
         [ 1,  1,  1,  ...,  9,  9,  9],
         [ 1,  1,  1,  ...,  9,  9,  9],
         ...,
         [29, 29, 29,  ..., 24, 24, 24],
         [29, 29, 29,  ..., 24, 24, 24],
         [29, 29, 29,  ..., 24, 24, 24]]], dtype=torch.uint8), 'instances': Instances(num_instances=10, image_heig

In [16]:
from detectron2.data import DatasetMapper

mapper = DatasetMapper(cfg, False)
test_data_loader = build_detection_test_loader(dataset=subsample_dataset.pool, mapper=mapper)

[02/19 16:50:02 d2.data.dataset_mapper]: [DatasetMapper] Augmentations used in inference: [ResizeShortestEdge(short_edge_length=(800, 800), max_size=1333, sample_style='choice')]


In [17]:
test_data_loader_iter = iter(test_data_loader)

In [18]:
print(next(test_data_loader_iter))

[{'file_name': '/public-dataset/BDD/bdd100k/images/100k/train/7b583008-124b6809.jpg', 'height': 720, 'width': 1280, 'image_id': 48271, 'image': tensor([[[189, 189, 189,  ..., 169, 169, 169],
         [189, 189, 189,  ..., 169, 169, 169],
         [189, 189, 189,  ..., 169, 169, 169],
         ...,
         [201, 201, 201,  ...,  65,  65,  65],
         [201, 201, 201,  ...,  64,  64,  64],
         [201, 201, 201,  ...,  64,  64,  64]],

        [[147, 147, 147,  ..., 130, 130, 130],
         [147, 147, 147,  ..., 130, 130, 130],
         [147, 147, 147,  ..., 130, 130, 130],
         ...,
         [219, 219, 219,  ...,  32,  32,  32],
         [219, 219, 219,  ...,  31,  31,  31],
         [219, 219, 219,  ...,  31,  31,  31]],

        [[110, 110, 110,  ...,  98,  98,  98],
         [110, 110, 110,  ...,  98,  98,  98],
         [110, 110, 110,  ...,  98,  98,  98],
         ...,
         [206, 206, 206,  ...,  36,  36,  36],
         [206, 206, 206,  ...,  35,  35,  35],
         [2

# Other stuff

In [ ]:
torch.manual_seed(0)
torch.rand(5)

In [ ]:
torch.rand(5)

In [ ]:
from detectron2.data import build_detection_train_loader
from detectron2.data.samplers import TrainingSampler

In [ ]:
dataloader1 = build_detection_train_loader(cfg)

sampler2 = TrainingSampler(size=len(dataloader1.dataset.dataset), seed=1234)

dataloader2 = build_detection_train_loader(cfg, sampler=sampler2)

sampler3 = TrainingSampler(size=4, seed=1234)

dataloader3 = build_detection_train_loader(cfg, sampler=sampler3)


In [ ]:
# To show that each is a different object
print("DataLoader 1 Memory Address: ")
print(dataloader1)
print("DataLoader 2 Memory Address: ")
print(dataloader2)
print("DataLoader 3 Memory Address: ")
print(dataloader3)
print("\n")

print("Dataset Lengths")
print("DataLoader 1: ", len(dataloader1.dataset.dataset))
print("DataLoader 2: ", len(dataloader2.dataset.dataset))
print("DataLoader 3: ", len(dataloader3.dataset.dataset))
print("\n")

print("Sampler Random Seeds")
print("DataLoader 1: ", dataloader1.dataset.sampler._seed)
print("DataLoader 2: ", dataloader2.dataset.sampler._seed)
print("DataLoader 3: ", dataloader3.dataset.sampler._seed)
print("\n")

print("Sampler Size")
print("DataLoader 1: ", dataloader1.dataset.sampler._size)
print("DataLoader 2: ", dataloader2.dataset.sampler._size)
print("DataLoader 3: ", dataloader3.dataset.sampler._size)
print("\n")

In [ ]:
print(torch.initial_seed())

In [ ]:
dataset = dataloader1.dataset.dataset

In [ ]:
dataset.__len__()

In [ ]:
dataloader2.dataset.dataset = dataset